In [1]:
from datetime import datetime
from num2words import num2words


In [4]:
import json
import pandas as pd
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder
import urllib.parse
headers = {
    "Origin": "www.mca.gov.in",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "X-Requested-With": "XMLHttpRequest",
}
#bipul's id
data="data=Ut8pBOc0RSM6iYqffqN1ovf7bobPRWJxrpoJRNCmK3GGtEoRKl3FZEf8xy36Iw3GHkoeOLH2Iw8tFFM6yB%2F6gZ4FgTyvicwoT%2FfDKJjUEuk9iiwbfqGT4bPey9LVPXwmJrnHl2AVXwPwjTY7BjtpUZ7CCRrYzb8G8Hs%2FTqLxp0oBRcfBtzGPnIVN9fMdkw%2Fj%2FSvMTL%2FNk05TbSN%2FPlQW%2FgdaTJRMuzmlIKp26ixHs8k%3D"
#srikanth's id
# data = "data=Ut8pBOc0RSM6iYqffqN1oiyj9WvJGiJEVVlWEbrO16gkiUbYKGssebXlM%2Bcc1l6%2FH53ORIVejkqRQNJO6whx4OCP6P3nRpnWDjVVIN5IRbmcNMmCFHf2ri7KqE4SRlNjYCw9u2LKROliTfvv63AV9eiPCiRWO5aLhpI2CVZau9pnzCFWwsAby4nwrZ19gu8G0zsW1uZyNhasaqAt7a%2F%2FLQ%3D%3D"
response = requests.post(
    "https://www.mca.gov.in/bin/mca/login", headers=headers, data=data)
set_cookie_value = response.headers.get('Set-Cookie')
start_index = set_cookie_value.find("session-token=") + len("session-token=")
end_index = set_cookie_value.find(";", start_index)
session_token = set_cookie_value[start_index:end_index]


In [30]:
with open('USER.json') as f:
    inp = json.load(f)
srn_value = inp['srn']

In [31]:
with open('CA.json') as f:
    inp1 = json.load(f)


In [32]:
def save(payload, session_token):
    url = 'https://www.mca.gov.in/bin/fillip/OnloadSubmit'
    headers = {
        'Host': 'www.mca.gov.in',
        'Cookie': f'cookiesession1=678B2869144E6B476733B93A4104896E;session-token={session_token}',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Origin': 'https://www.mca.gov.in',
    }

    payload1 = f"formData={json.dumps(payload)}"
    response = requests.post(url, headers=headers, data=payload1)
    print(response.text)
    return response


In [33]:
payload = {"requestBody": {"action": "onload",
                           "operation": "onload", "referenceNumber": "", "srn": "dfs"}}
payload['requestBody']['srn'] = srn_value
headers = {
    'Host': 'www.mca.gov.in',
    'Cookie': f'cookiesession1=678B28690FE85E2F44A6F01E56B5BE61; session-token={session_token}',
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    'Origin': 'https://www.mca.gov.in',
    'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html',
}
data = f"formData={json.dumps(payload)}"
response1 = requests.post(
    'https://www.mca.gov.in/bin/fillip/srnDetails', headers=headers, data=data)
c_name1 = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['proposedname1']
formData = "%7B%22requestBody%22%3A%7B%22operation%22%3A%22Save%22%2C%22integrationId%22%3A%221-ABC%22%2C%22referenceNumber%22%3Anull%2C%22srn%22%3A%22%22%2C%22formId%22%3A%22Homogeneous%22%2C%22formName%22%3A%22FiLLiP%22%2C%22formDescription%22%3A%22%22%2C%22formVersion%22%3A%221.2%22%2C%22userId%22%3A%22KASHVIJAIN19%40GMAIL.COM%22%2C%22CIN%22%3Anull%2C%22SRFOStatus%22%3A%22Draft%2FPending+Submission%22%2C%22formData%22%3A%7B%22formIntegrationId%22%3A%221-Test%22%2C%22isRegistered%22%3A%22Y%22%2C%22srn%22%3A%22M28067253%22%2C%22incorporationType%22%3A%22New+Incorporation%22%2C%22cin%22%3Anull%2C%22proposedName%22%3A%22INVITEFULFUSION+LLP%22%2C%22attachment%22%3Afalse%2C%22attachment2a%22%3Afalse%2C%22attachment2c%22%3Afalse%2C%22attachment2d%22%3Afalse%2C%22attachment2e%22%3Afalse%2C%22significanceProposedName%22%3Anull%2C%22vernacularLanguage%22%3Anull%2C%22isRegisteredPending%22%3Anull%2C%22particulars%22%3Anull%7D%2C%22formAttachment%22%3A%5B%5D%7D%7D"
decoded_data = urllib.parse.unquote_plus(formData, encoding='utf-8')
data_start = decoded_data.find('{')

data_end = decoded_data.rfind('}') + 1
data_str = decoded_data[data_start:data_end]
a = (json.loads((data_str)))
a['requestBody']['srn'] = ""
a['requestBody']['formData']['srn'] = srn_value
a['requestBody']['formData']['proposedName'] = c_name1
a['requestBody']['userId'] = inp1['userId']
response = save(a, session_token)
formintid = json.loads(json.loads(response.text)['resStr'])[
    'data']['formIntegrationId']
intid = json.loads(json.loads(response.text)['resStr'])[
    'data']['integrationId']
ref_no = json.loads(json.loads(response.text)['resStr'])[
    'data']['referenceNumber']


{"resCode":200,"resStr":"{\r\n\t\"error\": \"\",\r\n\t\"message\": \"Data Added Successfully\",\r\n\t\"data\": {\r\n\t\t\"integrationId\": \"1-54T2M2Z\",\r\n\t\t\"formIntegrationId\": \"1-54T2M3V\",\r\n\t\t\"referenceNumber\": \"1-11174607179\",\r\n\t\t\"srn\": \"\",\r\n\t\t\"dateTime\": \"\",\r\n\t\t\"srnStatus\":\"Draft/Pending Submission\",\r\n\t\t\"formType\":\"\"\r\n\t}\r\n}"}


In [34]:
response1.text

'{"resCode":200,"resStr":"{\\"error\\":\\"\\",\\"message\\":\\"Data Fetched Successfully\\",\\"data\\":{\\"status\\":\\"Approved\\",\\"reason\\":\\"\\",\\"formData\\":{\\"formIntegrationId\\":\\"1-4ZFF8RJ\\",\\"purpose\\":\\"New Incorporation\\",\\"cin\\":\\"\\",\\"llpin\\":\\"\\",\\"NICcode1\\":\\"52291\\",\\"NICcode2\\":\\"79120\\",\\"NICcode3\\":\\"79110\\",\\"NICCode1Desc\\":\\"Activities of travel agents and tour operators\\",\\"NICCode2Desc\\":\\"Tour operator activities\\",\\"NICCode3Desc\\":\\"Travel agency activities\\",\\"proposedname1\\":\\"VOYAGENICTRAVELS LLP\\",\\"proposedname2\\":\\"VOYAGERAGENIC LLP\\",\\"formAttachment\\":[{\\"attachmentLabel\\":\\"RUN LLP\\",\\"attachmentCategory\\":\\"Forms\\",\\"activityFileName\\":\\"M28186110.pdf\\",\\"activityFileSrcType\\":\\"I\\",\\"activityFileExt\\":\\"pdf\\",\\"activityFileSize\\":\\"27348\\",\\"activityComments\\":\\"Generated PDF Stored Successfully\\",\\"attachmentDMSId\\":\\"362099739\\",\\"totalNoOfPages\\":\\"1\\",\\"a

In [35]:
NICcode1 = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['NICcode1']
NICcode2 = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['NICcode2']
NICcode3 = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['NICcode3']
NICCode1Desc = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['NICCode1Desc']
NICCode2Desc = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['NICCode2Desc']
NICCode3Desc = json.loads(json.loads(response1.text)['resStr'])[
    'data']['formData']['NICCode3Desc']


In [36]:
def upload_file_from_url(session_token, file_url, ref_no, filename, folder, attachment_category, attachment_label):
    response = requests.get(file_url)
    if response.status_code != 200:
        print(f'Failed to download file from URL: {file_url}')
        return
    headers = {
        'Cookie': f'cookiesession1=678B286955776DB15177E7C622CE6403; alertPopup=true; session-token={session_token}',
        'Origin': 'https://www.mca.gov.in',
    }
    payload = {
        'file': (filename, response.content, 'image/jpeg'),
        'folder': folder,
        'filemetadata': f'{{"form":"FiLLiP","srn":"{ref_no}","documentType":"Attachment","module":"FiLLiP","userid":"FiLLiP","applicationId":"{ref_no}","attachmentCategory":"{attachment_category}","attachmentLabel":"{attachment_label}"}}'
    }
    url = 'https://www.mca.gov.in/bin/mca/dms/addservice'
    upload_response = requests.post(url, headers=headers, files=payload)
    if upload_response.status_code == 200:
        print('File uploaded successfully')
        print('Upload Response: ', upload_response.text)
        return upload_response.text
    else:
        print('Failed to upload the file')


In [37]:
inp


{'isRegistered': 'YES',
 'srn': 'M28186110',
 'registeredAddress': {'addressI': 'BMS College ofEngineering',
  'addressII': 'Bull Temple Road',
  'pin': '560019',
  'area': 'Gaviopuram Extension',
  'city': 'Bangalore South',
  'District': 'Bangalore',
  'state': 'Karnataka',
  'country': 'India',
  'mobile': '9319421653'},
 'contactDetails': {'phone': '',
  'mobile': '9319421653',
  'fax': '',
  'email': 'Sachuuuu1428@gmail.com',
  'company': ''},
 '3eAdressProofURL': 'https://legalnitiai.s3.ap-south-1.amazonaws.com/GST-reg/Electricity+bill+(1).pdf',
 '3eUtilityBillsURL': 'https://legalnitiai.s3.ap-south-1.amazonaws.com/GST-reg/Electricity+bill+(1).pdf',
 'BasicdetailsofDesignatedpartner1': {'placeOfBirthState1': 'Uttar Pradesh',
  'placeOfBirthDistrict1': 'Agra',
  'mobileNo1': '9319421653',
  'emailid1': 'Sachuuuu1428@gmail.com'},
 'occupationType1': 'Private Employment',
 'educationalQualification1': "Bachelor's degree",
 'Permanentaddress1': {'addressI1': 'ward no 9 gandhi nagar',

In [38]:
# Proof of Office address along with NOC, if applicable (Conveyance / Lease deed/Rent Agreement along with rent receipts)

s3_url = inp1['addressproofwithNOC']
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1]
attachment_label = "3eAdressProof"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
att_resp1 = response
# Copy of Utility Bills Attachment

s3_url = inp['3eUtilityBillsURL']
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1]
attachment_label = "3eUtilityBills"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
att_resp2 = response
# (B) Particulars of individual designated partners not having DIN/DPIN
# (iii) Whether the present residential address same as permanent residential address
# Proof of Identity attachment


s3_url = inp['ProofOfIdentity1']['17-AADHAR_CARD_UPLOAD']
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1].replace("+", " ")
attachment_label = "6bIdentityProof"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
att_resp3 = response
# (B) Particulars of individual designated partners not having DIN/DPIN
# (iii) Whether the present residential address same as permanent residential address
# Residential Proof attachment

s3_url = inp['ResidentialProof1']["18-RESIDENTIAL_PROOF_UPLOAD_LINK"]
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1].replace("+", ' ')
attachment_label = "6bResidentialProof"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
response
att_resp4 = response
# (B) Particulars of individual designated partners not having DIN/DPIN
# (iii) Whether the present residential address same as permanent residential address
# Proof of Identity attachment


s3_url = inp['ProofOfIdentity2']['27-AADHAR_CARD_UPLOAD']
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1].replace("+", " ")
attachment_label = "6bIdentityProof"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
att_resp5 = response
# (B) Particulars of individual designated partners not having DIN/DPIN
# (iii) Whether the present residential address same as permanent residential address
# Residential Proof attachment
s3_url = inp['ResidentialProof2']["28-RESIDENTIAL_PROOF_UPLOAD_LINK"]
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1].replace("+", ' ')
attachment_label = "6bResidentialProof"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
response
att_resp6 = response


File uploaded successfully
Upload Response:  {"attachmentLabel":"3eAdressProof","attachmentCategory":"Attachment","activityFileName":"Electricity+bill+(1).pdf","activityFileSrcPath":"180699644","activityFileSrcType":"I","activityFileExt":"pdf","activityFileSize":"294662","attachmentDMSId":"369328127","totalNoOfPages":"1","activityFileDate":"09/27/2023","publicDocument":"Y","activityComments":"180699644:3eAdressProof:Document added successfully.","folderName":"","folderId":"180699644","version":"","addedBy":"Fo User"}
File uploaded successfully
Upload Response:  {"attachmentLabel":"3eUtilityBills","attachmentCategory":"Attachment","activityFileName":"Electricity+bill+(1).pdf","activityFileSrcPath":"180699644","activityFileSrcType":"I","activityFileExt":"pdf","activityFileSize":"294662","attachmentDMSId":"369328145","totalNoOfPages":"1","activityFileDate":"09/27/2023","publicDocument":"Y","activityComments":"180699644:3eUtilityBills:Document added successfully.","folderName":"","folderId

In [39]:
# (B) Particulars of individual designated partners not having DIN/DPIN
# (i) Basic details of Designated partner
# Verify Pan
headers = {
    "Host": "www.mca.gov.in",
    "Cookie": f"cookiesession1=678B2869144E6B476733B93A4104896E; session-token={session_token}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.mca.gov.in",
}
# Extract values from inp dictionary, handling missing keys
pan_number = inp['ProofOfIdentity1'].get('17-PAN_CARD_NUMBER1', '')
first_name = inp['ProofOfIdentity1'].get('firstName1', '')
mid_name = inp['ProofOfIdentity1'].get('midName1', '')
surname = inp['ProofOfIdentity1'].get('surname1', '')
dob = inp['ProofOfIdentity1'].get('17-DATE', '')

# Format the date in the desired format
formatted_dob = datetime.strptime(
    dob.split('T')[0], "%Y-%m-%d").strftime("%d/%m/%Y")

# Concatenate first_name, mid_name, and surname to create fullName
full_name = f"{first_name} {mid_name} {surname}".strip()

# Create the payload dictionary
payload = {
    "panData": f'{{"pan": "{pan_number}","fName": "{first_name}","midName": "{mid_name}","surName": "{surname}","dob": "{formatted_dob}","fullName": "{full_name}"}}'
}
response = requests.post(
    "https://www.mca.gov.in/bin/cbdtPanValidate", headers=headers, data=payload)
if response.status_code == 200:
    print(response.text)
else:
    print(f"Error: {response.status_code} - {response.reason}")
pan_number = inp['ProofOfIdentity2'].get('17-PAN_CARD_NUMBER2', '')
first_name = inp['ProofOfIdentity2'].get('firstName2', '')
mid_name = inp['ProofOfIdentity2'].get('midName2', '')
surname = inp['ProofOfIdentity2'].get('surname2', '')
dob = inp['ProofOfIdentity2'].get('27-DATE', '')

# Format the date in the desired format
formatted_dob = datetime.strptime(
    dob.split('T')[0], "%Y-%m-%d").strftime("%d/%m/%Y")

# Concatenate first_name, mid_name, and surname to create fullName
full_name = f"{first_name} {mid_name} {surname}".strip()

# Create the payload dictionary
payload = {
    "panData": f'{{"pan": "{pan_number}","fName": "{first_name}","midName": "{mid_name}","surName": "{surname}","dob": "{formatted_dob}","fullName": "{full_name}"}}'
}
response = requests.post(
    "https://www.mca.gov.in/bin/cbdtPanValidate", headers=headers, data=payload)
if response.status_code == 200:
    print(response.text)
else:
    print(f"Error: {response.status_code} - {response.reason}")


{"resCode":200,"resStr":"{\"status\":\"true\"}"}
{"resCode":200,"resStr":"{\"status\":\"true\"}"}


In [40]:
if int(inp['individualNotValidDINorDPIN']) > 0:
    i = int(inp['individualNotValidDINorDPIN'])
    count = i  # Set the count based on the value of i
    indnotdin = []
    for j in range(1, count + 1):
        new_array = {
            'Category': 'Particulars of individual designated partners not having DIN/DPIN',
            'firstName': inp[f'ProofOfIdentity{j}'].get(
                f'firstName{j}', ''
            ),
            'middleName': inp[f'ProofOfIdentity{j}'].get(
                f'middleName{j}', ''
            ),
            'surname': inp[f'ProofOfIdentity{j}'].get(
                f'surname{j}', ''
            ),
            'fatherFirstName': inp[f'ProofOfIdentity{j}'].get(
                f'fatherFirstName{j}', ''
            ),
     'fatherMiddleName': inp[f'ProofOfIdentity{j}'].get(
                f'fatherMiddleName{j}', ''
            ),
     'fathersurname': inp[f'ProofOfIdentity{j}'].get(
                f'fathersurname{j}', ''
            ),
     'gender': 'Male',
     'dateofBirth': datetime.strptime((inp[f'ProofOfIdentity{j}'].get(f'{j}7-DATE', '')).split('T')[0], "%Y-%m-%d").strftime("%m/%d/%Y"),
     'nationality': 'India',
     'isResidentInIndia': 'Y',
     'isPanPassport': 'PAN',
     'panPassportDetails': inp[f'ProofOfIdentity{j}'].get(f'17-PAN_CARD_NUMBER{j}', ''),
     'placeOfBirthState': inp[f'BasicdetailsofDesignatedpartner{j}'][f'placeOfBirthState{j}'],
     'placeOfBirthDistrict': inp[f'BasicdetailsofDesignatedpartner{j}'][f'placeOfBirthDistrict{j}'],
     'partnerIsCitizenofIndia': 'Y',
     'occupationType': inp[f'occupationType{j}'],
     'descriptionOfOthers': None,
     'areaOfOccupation': inp[f'occupationType{j}'],
     'otherAreaOfOccupation': None,
     'educationalQualification': inp[f'educationalQualification{j}'],
     'otherEducation': None,
     'mobileNo': f"{urllib.parse.quote('+')}91{(inp[f'BasicdetailsofDesignatedpartner{j}'][f'mobileNo{j}'])}",
     'emailid': inp[f'BasicdetailsofDesignatedpartner{j}'][f'emailid{j}'],
     'proodOfIdentity': 'Aadhaar',
     'residentialproof': inp[f'ResidentialProof{j}'][f'{j}8-RESIDENTIAL_PROOF_OPTION'],
     'identityProofNo': inp[f'ProofOfIdentity{j}'][f'{j}7-AADHAR_CARD_NUMBER'],
     'residentialProofNo': inp[f'ResidentialProof{j}'][f'{j}8-RESIDENTIAL_PROOF_NUMBER'],
     'numberOfsharesHeld': None,
     'paidupvalueofsharesheld': None,
     'formOfContribution': inp[f'formOfContribution{j}'],
     'otherModeOfPayment': 'CHEQUE',
     'monetaryValueOfContributionInFigures': inp[f'monetaryValueOfContributionInFigures{j}'],
     'monetaryValueOfContributionInWords': ((num2words(int(inp[f'monetaryValueOfContributionInFigures{j}']), lang='en_IN').capitalize())),
     'numberOfLLPpartner': '0',
     'numberOfcompanyDirector': '0',
     'presentAddressSameAddressAsPremanent': 'Y',
     'durationStayedAtPresentAddressYears': None,
     'durationStayedAtPresentAddressMonth': None,
     'addressOfPreviousResidence': None,
     'designatedpartnerID': '',
     'name': '',
     'isResidentIndia': 'Y',
     'paid': '',
     'modeOfPay': inp[f'formOfContribution{j}'],
     'monetaryValueInFigures': inp[f'monetaryValueOfContributionInFigures{j}'],
     'monetaryValueInwords': ((num2words(int(inp[f'monetaryValueOfContributionInFigures{j}']), lang='en_IN').capitalize())),
     'numberOfLLP': '',
     'numberOfCompanyHoldAsDirector': '',
     'partnerFirstName':  inp[f'ProofOfIdentity{j}'].get(
                f'firstName{j}', ''
            ),
     'partnerMiddleName':  inp[f'ProofOfIdentity{j}'].get(
                f'middleName{j}', ''
            ),
     'partnerSurname':  inp[f'ProofOfIdentity{j}'].get(
                f'surname{j}', ''
            ),
     'partnerFatherFirstname': inp[f'ProofOfIdentity{j}'].get(
                f'fatherFirstName{j}', ''
            ),
     'partnerFatherMiddlename': inp[f'ProofOfIdentity{j}'].get(
                f'fatherMiddleName{j}', ''
            ),
     'partnerFatherSurname': inp[f'ProofOfIdentity{j}'].get(
                f'fathersurname{j}', ''
            ),
     'partnerGender': 'Male',
     'partnerDateofBirth': datetime.strptime((inp[f'ProofOfIdentity{j}'].get(f'{j}7-DATE', '')).split('T')[0], "%Y-%m-%d").strftime("%m/%d/%Y"),
     'partnerNationality': 'India',
     'partnerIsResidentIndia': 'Y',
     'isPanPass': 'PAN',
     'panPassDetails': inp[f'ProofOfIdentity{j}'].get(f'17-PAN_CARD_NUMBER{j}', ''),
     'partnerPlaceofBirthState': inp[f'BasicdetailsofDesignatedpartner{j}'][f'placeOfBirthState{j}'],
     'partnerPlaceofBirthDistrict': inp[f'BasicdetailsofDesignatedpartner{j}'][f'placeOfBirthDistrict{j}'],
     'OccupationType': inp[f'occupationType{j}'],
     'isresidentofIndia': 'Y',
     'numberofsharesheld': '',
     'passportOrPAN': 'PAN',
     'emailID': inp[f'BasicdetailsofDesignatedpartner{j}'][f'emailid{j}'],
     'previousResidence ': '',
     'numberOfSharesHeld': '',
     'PaidupvalueShares': '',
     'ListOfbodyParnerIndOfficeAddress': {'bodyParnerIndOfficeAddress': [{'addressType': 'Permanent Address',
        'addressI': inp[f'Permanentaddress{j}'][f'addressI{j}'],
        'addressII': inp[f'Permanentaddress{j}'][f'addressII{j}'],
        'country': inp[f'Permanentaddress{j}'][f'country{j}'],
        'pin': inp[f'Permanentaddress{j}'][f'pin{j}'],
        'area': inp[f'Permanentaddress{j}'][f'area{j}'],
        'city': inp[f'Permanentaddress{j}'][f'city{j}'],
        'district': inp[f'Permanentaddress{j}'][f'district{j}'],
        'state': inp[f'Permanentaddress{j}'][f'state{j}'],
        'policeStation': 'Hariharpur',
        'phone': inp[f'Permanentaddress{j}'][f'phone{j}']},
       {'addressType': 'Present Address',
        'addressI': inp[f'Permanentaddress{j}'][f'addressI{j}'],
        'addressII': inp[f'Permanentaddress{j}'][f'addressII{j}'],
        'country': inp[f'Permanentaddress{j}'][f'country{j}'],
        'pin': inp[f'Permanentaddress{j}'][f'pin{j}'],
        'area': inp[f'Permanentaddress{j}'][f'area{j}'],
        'city': inp[f'Permanentaddress{j}'][f'city{j}'],
        'district': inp[f'Permanentaddress{j}'][f'district{j}'],
        'state': inp[f'Permanentaddress{j}'][f'state{j}'],
        'policeStation': inp[f'Permanentaddress{j}'][f'area{j}'],
        'phone': inp[f'Permanentaddress{j}'][f'phone{j}']},
       {'addressType': 'Registered Address',
        'addressI': inp[f'Permanentaddress{j}'][f'addressI{j}'],
        'addressII': inp[f'Permanentaddress{j}'][f'addressII{j}'],
        'country': inp[f'Permanentaddress{j}'][f'country{j}'],
        'pin': inp[f'Permanentaddress{j}'][f'pin{j}'],
        'area': inp[f'Permanentaddress{j}'][f'area{j}'],
        'city': inp[f'Permanentaddress{j}'][f'city{j}'],
        'district': inp[f'Permanentaddress{j}'][f'district{j}'],
        'state': inp[f'Permanentaddress{j}'][f'state{j}'],
        'policeStation': 'Hariharpur',
        'phone': inp[f'Permanentaddress{j}'][f'phone{j}']}]}}
        indnotdin.append(new_array)


In [41]:
payload = {'requestBody': {'operation': 'Save',
                           'integrationId': intid,
                           'referenceNumber': ref_no,
                           'srn': '',
                           'formId': 'Homogeneous',
                           'formName': 'FiLLiP',
                           'formDescription': '',
                           'formVersion': '1.2',
                           'userId': inp1['userId'],
                           'CIN': None,
                           'SRFOStatus': 'Draft/Pending Submission',
                           'formData': {'formIntegrationId': formintid,
                                        'isRegistered': 'Y',
                                        'srn': srn_value,
                                        'incorporationType': 'New Incorporation',
                                        'cin': None,
                                        'proposedName': c_name1,
                                        'attachment': False,
                                        'attachment2a': False,
                                        'attachment2c': False,
                                        'attachment2d': False,
                                        'attachment2e': False,
                                        'significanceProposedName': None,
                                        'vernacularLanguage': None,
                                        'isRegisteredPending': None,
                                        'particulars': None,
                                        'registeredAddress': {'addressI': inp['registeredAddress']['addressI'],
                                                              'addressII': inp['registeredAddress']['addressII'],
                                                              'country': inp['registeredAddress']['country'],
                                                              'pin': inp['registeredAddress']['pin'],
                                                              'area': inp['registeredAddress']['area'],
                                                              'city': inp['registeredAddress']['city'],
                                                              'district': inp['registeredAddress']['District'],
                                                              'state': inp['registeredAddress']['state'],
                                                              'longitude': '73.859207',
                                                              'latitude': '18.528851',
                                                              'policeStation': 'banashankari',
                                                              'phone': None,
                                                              'mobile': inp['registeredAddress']['mobile'],
                                                              'fax': None,
                                                              'email': inp['contactDetails']['email'],
                                                              'registrarOfficeName': 'Registrar of Companies, Karnataka'},
                                        'buissnessactivity': [{'srno': '1',
                                                               'industrySubClass': 'Axxx010',
                                                               'description': 'abc rtd',
                                                               'primary': 'No'},
                                                              {'srno': '2',
                                                               'industrySubClass': 'bxxx010',
                                                               'description': 'fghty rtd',
                                                               'primary': 'yes'}],
                                        'mainIndustrialNICcode': NICcode1,
                                        'mainIndustrialDescriptionOfNIC': NICCode1Desc,
                                        'otherIndustrialNICcode': NICcode2,
                                        'otherIndustrialDescriptionOfNIC': NICCode2Desc,
                                        'NICCode3': NICcode3,
                                        'NICCode3Desc': NICCode3Desc,
                                        'numberOfDesignatedPartnersValidDIN_DPIN': '0',
                                        'numberOfDesignatedPartnersNotValidDIN_DPIN': str(inp['Number_of_Designated_partners']),
                                        'individualValidDINorDPIN': '0',
                                        'individualNotValidDINorDPIN': inp['individualNotValidDINorDPIN'],
                                        'nominiesValidDINorDPIN': '0',
                                        'nominiesNotValidDINorDPIN': '0',
                                        'numberOfPartnersOtherThanDesignatedPartnerValidDINorDPIN': '0',
                                        'numberOfPartnersOtherThanDesignatedPartnerNotValidDINorDPIN': '0',
                                        'individualOtherThanDesignatedPartnerValidDINorDPIN': '0',
                                        'individualOtherThanDesignatedPartnerNotValidDINorDPIN': '0',
                                        'nominiesOtherThanDesignatedPartnerValidDINorDPIN': '0',
                                        'nominiesOtherThanDesignatedPartnerNotValidDINorDPIN': '0',
                                        'totalNumberValidDesignatedPartners': '0',
                                        'totalNumberNotValidDesignatedPartners': str(inp['total_Number_Of_Partners']),
                                        'MCALLPFiLLiPIndPartner': indnotdin,
                           'formAttachment': []}}}


In [42]:
payload['requestBody']['formData']['formAttachment']=[att_resp1,att_resp2,att_resp3,att_resp4,att_resp5,att_resp6]

In [43]:
response=save(payload,session_token)

{"resCode":200,"resStr":"{\r\n\t\"error\": \"\",\r\n\t\"message\": \"Data Added Successfully\",\r\n\t\"data\": {\r\n\t\t\"integrationId\": \"1-54T2M2Z\",\r\n\t\t\"formIntegrationId\": \"1-54T2M3V\",\r\n\t\t\"referenceNumber\": \"1-11174607179\",\r\n\t\t\"srn\": \"\",\r\n\t\t\"dateTime\": \"\",\r\n\t\t\"srnStatus\":\"Draft/Pending Submission\",\r\n\t\t\"formType\":\"\"\r\n\t}\r\n}"}


In [44]:
# sourcery skip: avoid-builtin-shadow
sum=0
if int(inp['individualNotValidDINorDPIN']) > 0:
    i = int(inp['individualNotValidDINorDPIN'])
    count = i  # Set the count based on the value of i
    for j in range(1, count + 1):
        sum=sum+int(inp[f'monetaryValueOfContributionInFigures{j}'])

In [45]:
import requests

url = 'https://www.mca.gov.in/bin/fillip/fillipAoCode'

pan_params = {
    'AO_NO': "93",
    'AO_CODE': "P"+"KAR",
    'RANGE_CODE': "224",
    'AO_TYPE': "W"
}

headers = {
    'Cookie': f'cookiesession1=678B2869144E6B476733B93A4104896E; deviceId=kvkef960kw9; alertPopup=true; session-token={session_token}',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html?applicationHistory=eyJzcm4iOiIiLCJyZWZlcmVuY2UiOiIxLTEwMjgyMTE5NTA4IiwicHVycG9zZSI6Ik5ldyBJbmNvcnBvcmF0aW9uIn0%3D',
}

response = requests.get(url, params=pan_params, headers=headers)

if response.status_code == 200:
    print("Request successful!")
    print("Response text:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")

pan_valid=json.loads(json.loads(response.text)['resStr'])['data']['is_valid']


Request successful!
Response text:
{"resCode":200,"resStr":"{\n        \t\t\n        \t\t\"message\":\"Data fetched Successfully\",\n        \t\t\"data\":{\"is_valid\":1}\n        \t}"}


In [46]:
pan_params

{'AO_NO': '93', 'AO_CODE': 'PKAR', 'RANGE_CODE': '224', 'AO_TYPE': 'W'}

In [47]:
# import requests
# import pandas as pd
# import json
# from concurrent.futures import ThreadPoolExecutor

# # Function to process a single row
# def process_row(row):
#     global count_is_valid_1
#     global count_skipped

#     if row['AREA CODE'] == 'KAR':
#         pan_params = {
#             'AO_NO': str(row['AO NO.']),
#             'AO_CODE': "P"+str(row['AREA CODE']),
#             'RANGE_CODE': str(row['RANGE CODE']),
#             'AO_TYPE': str(row['AO TYPE'])
#         }

#         response = requests.get(url, params=pan_params, headers=headers)

#         if response.status_code == 200:
#             pan_data = json.loads(json.loads(response.text)['resStr'])['data']
#             isValid = pan_data.get('is_valid', 0)

#             if isValid == 1:
#                 count_is_valid_1 += 1
#                 print(f"Request {row.name + 1}:")
#                 print(f"  'AO_NO': {pan_params['AO_NO']}")
#                 print(f"  'AO_CODE': {pan_params['AO_CODE']}")
#                 print(f"  'RANGE_CODE': {pan_params['RANGE_CODE']}")
#                 print(f"  'AO_TYPE': {pan_params['AO_TYPE']}")
#                 print("  isValid is 1.")
#         else:
#             print(f"Request {row.name + 1} failed with status code: {response.status_code}")
#     else:
#         count_skipped += 1

# # Load the Excel file into a DataFrame
# df = pd.read_excel(r'2023-002-TIN-Annexure_1-Release_PAN_AO_code_Master_Other_than_Mumbai_v.9.2.xlsx')

# # Define the URL and headers
# url = 'https://www.mca.gov.in/bin/fillip/fillipAoCode'

# headers = {
#     'Cookie': f'cookiesession1=678B2869144E6B476733B93A4104896E; deviceId=kvkef960kw9; alertPopup=true; session-token={session_token}',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.5790.110 Safari/537.36',
# }

# isValid = 0  # Initialize isValid to 0
# count_is_valid_1 = 0  # Initialize a counter for isValid = 1
# count_skipped = 0  # Initialize a counter for skipped rows

# # Use ThreadPoolExecutor for concurrent processing
# with ThreadPoolExecutor(max_workers=5) as executor:
#     futures = [executor.submit(process_row, row) for _, row in df.iterrows()]

# # Wait for all futures to complete
# for future in futures:
#     future.result()

# # Calculate the count of rows that were not processed or skipped
# count_not_available = len(df) - count_is_valid_1 - count_skipped

# print(f"Total number of times 'isValid' is 1 for rows with 'AREA CODE' as 'KAR': {count_is_valid_1}")
# print(f"Total number of rows skipped (not processed): {count_skipped}")
# print(f"Total number of rows that aren't available: {count_not_available}")


In [48]:
# pan_valid_list

In [49]:
import requests

url = 'https://www.mca.gov.in/bin/fillip/fillipAoCode'

tan_params = {
    'AO_NO': "3",
    'AO_CODE': "T"+"KAR",
    'RANGE_CODE': "182",
    'AO_TYPE':"WT"
}

headers = {
    'Cookie': f'cookiesession1=678B2869144E6B476733B93A4104896E; session-token={session_token}',
    'Referer': 'https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html?applicationHistory=eyJzcm4iOiIiLCJyZWZlcmVuY2UiOiIxLTEwMjgyMTE5NTA4IiwicHVycG9zZSI6Ik5ldyBJbmNvcnBvcmF0aW9uIn0%3D',
}

response = requests.get(url, params=tan_params, headers=headers)

if response.status_code == 200:
    print("Request successful!")
    print("Response text:")
    print(response.text)
else:
    print(f"Request failed with status code: {response.status_code}")
tan_valid=json.loads(json.loads(response.text)['resStr'])['data']['is_valid']

Request successful!
Response text:
{"resCode":200,"resStr":"{\n        \t\t\n        \t\t\"message\":\"Data fetched Successfully\",\n        \t\t\"data\":{\"is_valid\":1}\n        \t}"}


In [50]:
payload['requestBody']['formData']["presentAddressSameAddressAsPremanent"]=''
payload['requestBody']['formData']["durationStayedAtPresentAddressYears"]=''
payload['requestBody']['formData']["durationStayedAtPresentAddressMonth"]=''
payload['requestBody']['formData']["addressOfPreviousResidence"]=''
payload['requestBody']['formData']["proodOfIdentity"]=''
payload['requestBody']['formData']["identityProofNo"]=''
payload['requestBody']['formData']["residentialproof"]=''
payload['requestBody']['formData']["residentialProofNo"]=''
payload['requestBody']['formData']["numberOfSharesHeld"]=''
payload['requestBody']['formData']["paidSharesHeld"]=''
payload['requestBody']['formData']["otherModeOfPayment"]=''
payload['requestBody']['formData']["monetaryValueOfContributionInFigures"]=str(sum)
payload['requestBody']['formData']["monetaryValueOfContributionInWords"]=str((num2words(sum, lang='en_IN').capitalize()))
payload['requestBody']['formData']["numberOfLLPpartner"]=''
payload['requestBody']['formData']["numberOfcompanyDirector"]=''
payload['requestBody']['formData']["deductionAccounNumberTAN"]=''
payload['requestBody']['formData']["panAreacode"]='KAR'
payload['requestBody']['formData']["panAOtype"]='W'
payload['requestBody']['formData']["panRangecode"]='224'
payload['requestBody']['formData']["panAONo"]='93'
payload['requestBody']['formData']["tanAreacode"]='KAR'
payload['requestBody']['formData']["tanAOtype"] = 'WT'
payload['requestBody']['formData']["tanRangecode"]='182'
payload['requestBody']['formData']["tanAONo"]='3'
payload['requestBody']['formData']["sourceofIncome"]='Income from Business/profession '
payload['requestBody']['formData']["BusinessProfessionCode"]=inp1['BusinessProfessionCode']


In [52]:
response=save(payload,session_token)

{"resCode":200,"resStr":"{\r\n\t\"error\": \"\",\r\n\t\"message\": \"Data Added Successfully\",\r\n\t\"data\": {\r\n\t\t\"integrationId\": \"1-54T2M2Z\",\r\n\t\t\"formIntegrationId\": \"1-54T2M3V\",\r\n\t\t\"referenceNumber\": \"1-11174607179\",\r\n\t\t\"srn\": \"\",\r\n\t\t\"dateTime\": \"\",\r\n\t\t\"srnStatus\":\"Draft/Pending Submission\",\r\n\t\t\"formType\":\"\"\r\n\t}\r\n}"}


In [28]:
payload

{'requestBody': {'operation': 'Save',
  'integrationId': '1-54T0LML',
  'referenceNumber': '1-11174513277',
  'srn': '',
  'formId': 'Homogeneous',
  'formName': 'FiLLiP',
  'formDescription': '',
  'formVersion': '1.2',
  'userId': 'BIPULKUMARSINGH6690.COM',
  'CIN': None,
  'SRFOStatus': 'Draft/Pending Submission',
  'formData': {'formIntegrationId': '1-54T0LMN',
   'isRegistered': 'Y',
   'srn': 'M28186110',
   'incorporationType': 'New Incorporation',
   'cin': None,
   'proposedName': 'VOYAGENICTRAVELS LLP',
   'attachment': False,
   'attachment2a': False,
   'attachment2c': False,
   'attachment2d': False,
   'attachment2e': False,
   'significanceProposedName': None,
   'vernacularLanguage': None,
   'isRegisteredPending': None,
   'particulars': None,
   'registeredAddress': {'addressI': 'BMS College ofEngineering',
    'addressII': 'Bull Temple Road',
    'country': 'India',
    'pin': '560019',
    'area': 'Gaviopuram Extension',
    'city': 'Bangalore South',
    'district'

In [94]:
#validation certificate
s3_url = inp1['validation_cert']
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1].replace("+", ' ')
attachment_label = "valuationCert10a"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
response
att_resp7 = response

File uploaded successfully
Upload Response:  {"attachmentLabel":"valuationCert10a","attachmentCategory":"Attachment","activityFileName":"ilovepdf_merged+(1).pdf","activityFileSrcPath":"180225693","activityFileSrcType":"I","activityFileExt":"pdf","activityFileSize":"621830","attachmentDMSId":"367581030","totalNoOfPages":"2","activityFileDate":"09/21/2023","publicDocument":"Y","activityComments":"180225693:valuationCert10a:Document added successfully.","folderName":"","folderId":"180225693","version":"","addedBy":"Fo User"}


In [95]:
#validation certificate
s3_url = inp1['subscriber_sheet']
folder = ""
attachment_category = s3_url.split('/')[-1].split('.')[-1].replace("+", ' ')
attachment_label = "subscriberConsent"
filename = s3_url.split('/')[-1]
response = json.loads(upload_file_from_url(session_token, s3_url,
                      ref_no, filename, folder, attachment_category, attachment_label))
response['folderName'] = response['folderId']
response
att_resp8 = response

File uploaded successfully
Upload Response:  {"attachmentLabel":"subscriberConsent","attachmentCategory":"Attachment","activityFileName":"Scan2023-09-12_162313.pdf","activityFileSrcPath":"180225693","activityFileSrcType":"I","activityFileExt":"pdf","activityFileSize":"4063029","attachmentDMSId":"367581073","totalNoOfPages":"3","activityFileDate":"09/21/2023","publicDocument":"Y","activityComments":"180225693:subscriberConsent:Document added successfully.","folderName":"","folderId":"180225693","version":"","addedBy":"Fo User"}


In [96]:
payload['requestBody']['formData']['formAttachment'].append(att_resp7)
payload['requestBody']['formData']['formAttachment'].append(att_resp8)


In [97]:
headers = {
        'Host': 'www.mca.gov.in',
        'Cookie': f'cookiesession1=678B2869144E6B476733B93A4104896E;session-token={session_token}',
        'Origin': 'https://www.mca.gov.in',
    }
response2=requests.get("https://www.mca.gov.in/bin/mca/userEditDetails",headers=headers)

In [98]:
payload['requestBody']['formData']["dsc"]=''
payload['requestBody']['formData']["DIN"]=inp['ProofOfIdentity2']['17-PAN_CARD_NUMBER2']
payload['requestBody']['formData']["name"]=inp1['name']
payload['requestBody']['formData']["decProfessionalFatherName"]=inp1['fatherName']
payload['requestBody']['formData']["relation"] = "Son" if inp1['gender'] == "M" else "Daughter"
payload['requestBody']['formData']["profession"]='Chartered Accountant in whole time practice'
payload['requestBody']['formData']["practiceName"]='Institute of Chartered Accountants of India'
payload['requestBody']['formData']["practiceNumber"]=inp1['practiceNumber']
payload['requestBody']['formData']["regulatoryBody"]='Institute of Chartered Accountants of India'
payload['requestBody']['formData']["associate_fellow"]=inp1['associate_fellow']
payload['requestBody']['formData']["dsc2"]=''

In [99]:
response=save(payload,session_token)

{"resCode":200,"resStr":"{\r\n\t\"error\": \"\",\r\n\t\"message\": \"Data Added Successfully\",\r\n\t\"data\": {\r\n\t\t\"integrationId\": \"1-53H17AS\",\r\n\t\t\"formIntegrationId\": \"1-53H17AU\",\r\n\t\t\"referenceNumber\": \"1-11093919796\",\r\n\t\t\"srn\": \"\",\r\n\t\t\"dateTime\": \"\",\r\n\t\t\"srnStatus\":\"Draft/Pending Submission\",\r\n\t\t\"formType\":\"\"\r\n\t}\r\n}"}


In [100]:
payload['requestBody']['operation']="Submit"
response_text = save(payload,session_token)

{"resCode":200,"resStr":"{\r                 \n    \"membershipData\":{\r\n\"membershipFailure\":\"N\",\r\n\"error\":\"\",\r\n\"message\": \"Data fetched Successfully\"\r\n},\n    \"professionData\":{\n\t\t\t\t\t\t\t\"professionalPracticeFailure\": \"N\",\n\t\t\t\t\t\t\t\"error\": \"\",\n\t\t\t\t\t\t\t\"message\": \"NA\"\n\t\t\t\t\t\t\t},   \n    \"cloudEraData\":{\n  \"validationResponce\": {\n    \"validationresposeBody\": [], \n    \"submissionRestricted\": \"false\"\n  }, \n  \"message\": \"Auto check Completed\"\n}\n,                      \n\t\"validationResponse\": {\"SubmissionRestricted\":\"Y\",\"validationData\":[{\"alertType\":\"\",\"alertDescription\":\"\",\"alertCategory\":\"\"},{\"alertType\":\"\",\"alertDescription\":\"\",\"alertCategory\":\"\"},{\"alertType\":\"\",\"alertDescription\":\"\",\"alertCategory\":\"\"},{\"alertType\":\"\",\"alertDescription\":\"\",\"alertCategory\":\"\"},{\"alertType\":\"\",\"alertDescription\":\"\",\"alertCategory\":\"\"},{\"alertType\":\"\",

In [ ]:
import requests

# Define the URL and headers
url = "https://www.mca.gov.in/bin/fillip/OnloadSubmit"
headers = {
    "Host": "www.mca.gov.in",
    "Cookie": f"cookiesession1=678B2869144E6B476733B93A4104896E; session-token={session_token}",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://www.mca.gov.in",
    }
payload = {
    "formData": json.dumps({
        "requestBody": {
            "operation": "Load",
            "referenceNumber": ref_no,
            "srn": "",
            "formname": "FiLLiP"
        }
    })
}

# Send the POST request
response = requests.post(url, headers=headers, data=payload)

# Check the response
if response.status_code == 200:
    print("Request was successful.")
    print("Response content:")
    print(response.text)
else:
    print(f"Request failed with status code {response.status_code}:")
    print(response.text)

In [ ]:
nvd_details=json.loads(json.loads(response.text)['resStr'])['data']['formData']['particulatOfNotVaidDIN_DPIN']
f_details=json.loads(json.loads(response.text)['resStr'])['data']['MCALinkedFormDetails']

In [ ]:
f9_py={'requestBody': {'serveAction': 'saveForm',
  'operation': 'Submit',
  'integrationId': '1-ABC',
  'referenceNumber': '1-ABC',
  'srn': '',
  'formId': '',
  'formName': 'LLP Form 9',
  'formDescription': '',
  'formVersion': '2',
  'userId': inp1['userId'],
  'boWorkStatus': '',
  'mcaComments': '',
  'SRFOStatus': 'Draft/Pending Submission',
  'fiLLiPReferenceNumber': ref_no,
  'formData': {'formIntegrationId': '1-xuvw',
   'LLPName': c_name1,
   'addressOfLLP': 'RVCE Educational Institution Banashankari III Stage Bangalore South 560085 Karnataka India',
   'addressLongitude': '',
   'addressPolice': '',
   'addressState': 'Karnataka',
   'addressLatitude': '',
   'addressCity': 'Bangalore South',
   'addressAreaLocality': 'Banashankari III Stage',
   'addressLine1': 'RVCE Educational Institution Banashankari III Stage Bangalore South 560085 Karnataka India',
   'addressCountry': 'India',
   'addressPincode': '560085',
   'addressDistrict': '',
   'totalNoOfPartners': '0',
   'individuals': '0',
   'nomineesOfBodyCorporate': '0',
   'consentDesignationPartner': '0',
   'designatedPartnerDetails': [{'partnerIDNo': None,
     'nameOfPartner': None,
     'nameOfbodyCorporate': None,
     'registrationNumber': None}],
   'registrationNumber': '',
   'dsc': ''}}}